In [2]:
from neuron import h, gui

ImportError: dlopen(/Users/lampochka/miniconda2/lib/python2.7/site-packages/neuron/hoc.so, 2): Library not loaded: libreadline.6.2.dylib
  Referenced from: /Users/lampochka/miniconda2/lib/python2.7/site-packages/neuron/hoc.so
  Reason: image not found

In [43]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [44]:
h.load_file("Spencer.hoc")

1.0

In [ ]:
h.qui

In [46]:
from collections import defaultdict

In [ ]:
# Посчитатель всех токов:

def net_dendrite_currents():
    "Total dendrite currents in nA"
    net_i = defaultdict(lambda: 0)

    for i in range(int(h.Nseg)):
        dend = h.dend[i]
        for loc in dend:
            a = loc.area()*1e-8*1e6 #конвертируем площадь в см^2 и ток в nA заодно
            net_i['ik'] += loc.ik*a 
            net_i['ih'] += loc.hcno.ih*a
            net_i['ileak'] += loc.i_leak*a
    return net_i

def net_soma_currents():
    net_i = defaultdict(lambda: 0)
    loc = h.soma(0.5)
    a = loc.area()*1e-8*1e6 #конвертируем площадь в см^2 и ток в nA заодно
    net_i['ik'] += loc.ik*a # все К токи
    try:
        net_i['ina'] += loc.nas.ina*a
    except NameError:
        print 'skipping iNa'
        
    #net_i['ih'] += loc.hs.ih*a
    net_i['ih'] += loc.hcno.i*a
    net_i['ileak'] += loc.i_leak*a
    return net_i

def net_axon_currents():
    net_i = defaultdict(lambda: 0)
    for comp in (h.initial, h.axon):
        for loc in comp:
            a = loc.area()*1e-8*1e6 #конвертируем площадь в см^2 и ток в nA заодно
            #net_i['ik'] += loc.ik*a # все К токи
            #net_i['ih'] += loc.ihsab.ih*a
            #net_i['ih'] += loc.hcno.ih*a
            net_i['ileak'] += loc.i_leak*a
            net_i['ina'] += loc.nas.ina*a
    return net_i

In [1]:
# %%writefile Energy_counter.py
# TODO: во все компартменты добавить *все* токи, только ненужные токи "выключить",
#       выставив gbar=0


# Посчитатель энергии в равновесии: (а) Способом Moujahid et al., 2011,
#(б) Способом подсчета ионов Attwell & Laughlin 2001

# (а)

def net_section_currendenrg(section):
    "Sum up specific currents within section and return in nA"
    net_i = defaultdict(lambda: 0)
    "e_acc holds resting energies"
    e_acc = defaultdict(lambda: 0)
    Erevs = {'ina':h.initial.ena,
             'ik':h.soma.ek,
             'ih':h.soma.eh_hcno,
             'leak': h.soma.erev_leak
            }
    for seg in section:
        a = seg.area()*1e-8*1e6 # конвертируем площадь в см^2 и ток в nA заодно
        
        try: 
            net_i['ik'] += seg.ik*a # все К токи: Iklt, Ikht. nA
            e_acc['ik'] += seg.ik*a*(seg.v-Erevs['ik']) # nA*mV = pW = 1e-12*W = 1e-12*J/s
        except NameError: 
            print "Skipping iK"
    
        try: 
            net_i['ina'] += seg.ina*a 
            e_acc['ina'] += seg.ina*a*(seg.v-Erevs['ina'])
        except NameError: 
            print "Skipping iNa"
            
        try: 
            net_i['ih'] += seg.hcno.i*a 
            e_acc['ih'] += seg.hcno.i*a*(seg.v-Erevs['ih'])
        except NameError: 
            print "Skipping ih"
            
        try: 
            net_i['leak'] += seg.leak.i*a 
            e_acc['leak'] += seg.leak.i*a*(seg.v-Erevs['leak'])
        except NameError: 
            print "Skipping leak"  
    return net_i, e_acc

# (б)

def net_section_AtLau_enrg(section) :
    "Counts total resting energy consumption in ATP_moles/s (CHECK THIS in AL&Lau2001)"
    
    F = 96450 # Кл/млоь
    Rin = 6e9 # значение Rin из литературы (для крысы), Ом
    Fatp = 50*1e15 # энергия гидролиза АТФ в пДж/моль, по Moujahid et al., 2014 (п. 2.3)
    
#     Что делать с входным сопротивлением? По авторам формулы, Rin = 1/(gna+gk). 
#     Можно посчитать Rin в модели, но будет ли оно соответствовать формуле
#     Или: Какие брать значения gna, gk?
#     Вот, попробовал 6 МОм
  
    Erevs = {'ina':h.initial.ena,
             'ik':h.soma.ek,
             'ih':h.soma.eh_hcno,
             'leak': h.soma.erev_leak
            }
    for seg in section:
        J = ((Erevs['ina']-seg.v)*(seg.v-Erevs['ik']))/(F*Rin*(seg.v+2*Erevs['ina']-3*Erevs['ik']))
        # J в моль_АТФ/с
        
#     print  ("ATP_mol/s at rest --> %.3e" % (J))
#     print  ("ATPs/s at rest --> %.3e" % (J*6.02*1e23))
    return  ( "Energy in pJ/s %.6g" % (J*Fatp))

Writing Energy_counter.py


In [126]:
# Равновесная энергия сомы по Attwell & Laughlin 2001
# пДж/с

de = net_section_AtLau_enrg(h.dend[1])
se = net_section_AtLau_enrg(h.soma)
ie = net_section_AtLau_enrg(h.initial)

curs = ['ik','ina', 'ileak', 'ih']

print de,"\n", se,"\n", ie

Energy in pJ/s 370.467 
Energy in pJ/s 393.736 
Energy in pJ/s 392.221


In [87]:
# Равновесные токи и энергия сомы по Moujahid et al., 2011
# Единицы измерения: ток в нА, энергия в пДж/с
net_section_currendenrg(h.soma)

(defaultdict(<function __main__.<lambda>>,
             {'ih': -0.9293380660991897,
              'ik': 0.6457887984615786,
              'ina': 0.0,
              'leak': 0.09221122921554724}),
 defaultdict(<function __main__.<lambda>>,
             {'ih': 20.121897678519368,
              'ik': 6.682712911734947,
              'ina': 0.0,
              'leak': 0.21652487788671324}))

In [94]:
all_sections = [h.soma, h.axon, h.initial]  + [h.dend[i] for i in range(4)]


# total_current = map(net_section_current, all_sections)
total_current = [net_section_currendenrg(sec) for sec in all_sections]
all_values = [netc.values() for netc in total_current]

AttributeError: 'tuple' object has no attribute 'values'